In [ ]:
import './../../loadenv.mjs'

# helper utilities

In [6]:
import { ChatPromptTemplate, MessagesPlaceholder } from '@langchain/core/prompts'
import { StructuredTool } from '@langchain/core/tools'
import { convertToOpenAITool } from '@langchain/core/utils/function_calling'
import { Runnable } from '@langchain/core/runnables'
import { getModel } from './../../utils.mjs'
import { ChatOpenAI } from '@langchain/openai'

async function createAgent({
    llm,
    tools,
    systemMessage
}: {
    llm: ChatOpenAI;
    tools: StructuredTool[];
    systemMessage: string;
}): Promise<Runnable> {
    const toolNames = tools.map(tool => tool.name).join(', ')
    const formattedTools = tools.map(t => convertToOpenAITool(t))

    let prompt = ChatPromptTemplate.fromMessages([
        [
            "system",
            "You are a helpful AI assistant, collaborating with other assistants." +
            " Use the provided tools to progress towards answering the question." +
            " If you are unable to fully answer, that's OK, another assistant with different tools " +
            " will help where you left off. Execute what you can to make progress." +
            // " If you or any of the other assistants have the final answer or deliverable," +
            // " prefix your response with FINAL ANSWER so the team knows to stop." +
            /// 根据豆包的功能，对提示词进行修改
            " Only if you have all ready displayed a bar chart to user, " +
            " can you prefix your response with FINAL ANSWER so the team knows to stop." +

            " You have access to the following tools: {tool_names}.\n{system_message}",
        ],
        new MessagesPlaceholder("messages"),
    ])
    prompt = await prompt.partial({
        system_message: systemMessage,
        tool_names: toolNames,
    })

    return prompt.pipe(llm.bind({ tools: formattedTools }))
}

# define state

In [7]:
import { BaseMessage } from '@langchain/core/messages'
import { Annotation } from '@langchain/langgraph'

const AgentState = Annotation.Root({
    messages: Annotation<BaseMessage[]>({
        reducer: (x, y) => x.concat(y),
    }),
    sender: Annotation<string>({
        reducer: (x, y) => y ?? x ?? 'user',
        default: () => 'user',
    }),
})

# define tools

In [8]:
import { TavilySearchResults } from '@langchain/community/tools/tavily_search'
import { tool } from '@langchain/core/tools'
import * as d3 from 'd3'
import { createCanvas } from 'canvas'
import { z } from 'zod'
import { displayImage } from '../../utils.mjs'

const chartTool = tool(
    async ({ data }) => {
        const width = 500
        const height = 500
        const margin = { top: 20, right: 30, bottom: 30, left: 40 }
        
        const canvas = createCanvas(width, height)
        const ctx = canvas.getContext('2d')

        const x = d3
            .scaleBand()
            .domain(data.map(d => d.label))
            .range([margin.left, width - margin.right])
            .padding(0.1)
        const y = d3
            .scaleLinear()
            .domain([0, d3.max(data, (d: any) => d.value) ?? 0])
            .nice()
            .range([height - margin.bottom, margin.top])
        
        const colorPalette = [
            "#e6194B",
            "#3cb44b",
            "#ffe119",
            "#4363d8",
            "#f58231",
            "#911eb4",
            "#42d4f4",
            "#f032e6",
            "#bfef45",
            "#fabebe",
        ]
        data.forEach((d, idx) => {
            ctx.fillStyle = colorPalette[idx % colorPalette.length]
            ctx.fillRect(
                x(d.label) ?? 0,
                y(d.value),
                x.bandwidth(),
                height - margin.bottom - y(d.value),
            )
        })

        ctx.beginPath()
        ctx.strokeStyle = 'black'
        ctx.moveTo(margin.left, height - margin.bottom)
        ctx.lineTo(width - margin.right, height - margin.bottom)
        ctx.stroke()

        ctx.textAlign = 'center'
        ctx.textBaseline = 'top'
        x.domain().forEach((d: any) => {
            const xCoord = (x(d) ?? 0) + x.bandwidth() / 2
            ctx.fillText(d, xCoord, height - margin.bottom + 6)
        })

        ctx.beginPath()
        ctx.moveTo(margin.left, height - margin.top)
        ctx.lineTo(margin.left, height - margin.bottom)
        ctx.stroke()

        ctx.textAlign = 'right'
        ctx.textBaseline = 'middle'
        const ticks = y.ticks()
        ticks.forEach((d: any) => {
            const yCoord = y(d)
            ctx.moveTo(margin.left, yCoord)
            ctx.lineTo(margin.left - 6, yCoord)
            ctx.stroke()
            ctx.fillText(d.toString(), margin.left - 8, yCoord)
        })
        console.log('+++ show canvas png')
        const buffer = canvas.toBuffer()
        // 这个方法很重要
        await displayImage(buffer)
        return 'Chart has been generated and displayed to the user!'
    },
    {
        name: 'generate_bar_chart',
        description: 'Generates a bar chart from an array of data points using D3.js and displays it for the user.',
        schema: z.object({
            data: z
                .object({
                    label: z.string(),
                    value: z.number(),
                }).array(),
        })
    }
)

const tavilyTool = new TavilySearchResults()

# Create Graph

## define agent nodes

In [9]:
import { HumanMessage } from '@langchain/core/messages'
import type { RunnableConfig } from '@langchain/core/runnables'

async function runAgentNode(props: {
    state: typeof AgentState.State;
    agent: Runnable;
    name: string;
    config?: RunnableConfig;
}) {
    const { state, agent, name, config } = props
    let result = await agent.invoke(state, config)
    if (!result?.tool_calls || result.tool_calls.length === 0) {
        result = new HumanMessage({ ...result, name, })
    }
    return {
        messages: [result],
        sender: name,
    }
}

const llm = getModel()

const researchAgent = await createAgent({
    llm,
    tools: [tavilyTool],
    systemMessage: 'You should provide accurate data for the chart generator to use.\nLet\'s think step by step.'
})

async function researchNode(
    state: typeof AgentState.State,
    config?: RunnableConfig,
) {
    return runAgentNode({
        state,
        agent: researchAgent,
        name: 'Researcher',
        config,
    })
}

const chartAgent = await createAgent({
    llm,
    tools: [chartTool],
    systemMessage: 'Any charts you display will be visible by the user.'
})

async function chartNode(state: typeof AgentState.State) {
    return runAgentNode({
        state,
        agent: chartAgent,
        name: 'ChartGenerator',
    })
}

In [10]:
// const researchResults = await researchNode({
//     messages: [new HumanMessage('Research the US primaries in 2024')],
//     sender: 'User',
// })

// researchResults

## define tool node

In [11]:
import { ToolNode } from '@langchain/langgraph/prebuilt'

const tools = [tavilyTool, chartTool]
const toolNode = new ToolNode<typeof AgentState.State>(tools)

In [12]:
// await toolNode.invoke(researchResults)

## define edge logic

In [13]:
import { AIMessage } from '@langchain/core/messages'

function router(state: typeof AgentState.State) {
    const messages = state.messages
    const lastMessage = messages[messages.length - 1] as AIMessage
    if (lastMessage?.tool_calls && lastMessage.tool_calls.length > 0) {
        return 'call_tool'
    }
    if (typeof lastMessage.content === 'string' && lastMessage.content.startsWith('FINAL ANSWER')) {
        return 'end'
    }
    return 'continue'
}

## define the graph

In [ ]:
import { END, START, StateGraph } from '@langchain/langgraph'

const workflow = new StateGraph(AgentState)
    .addNode('Researcher', researchNode)
    .addNode('ChartGenerator', chartNode)
    .addNode('call_tool', toolNode)

workflow.addConditionalEdges(
    'Researcher',
    router,
    {
        continue: 'ChartGenerator',
        call_tool: 'call_tool',
        end: END,
    },
)

workflow.addConditionalEdges(
    'ChartGenerator',
    router,
    {
        continue: 'Researcher',
        call_tool: 'call_tool',
        end: END
    }
)

workflow.addConditionalEdges(
    'call_tool',
    x => x.sender,
    {
        Researcher: 'Researcher',
        ChartGenerator: 'ChartGenerator',
    }
)

workflow.addEdge(START, 'Researcher')
const graph = workflow.compile()

In [ ]:
const representation = graph.getGraph()
const image = await representation.drawMermaidPng()
const arrayBuffer = await image.arrayBuffer()

Deno.jupyter.image(new Uint8Array(arrayBuffer))

# Invoke

In [ ]:
const streamResults = await graph.stream(
    {
        messages: [
            new HumanMessage({
                content: 'Generate a bar chart of the US gdp over the past 3 years.'
            }),
        ],
    },
    {
        recursionLimit: 150,
    }
)

const prettifyOutput = (output: Record<string, any>) => {
    const keys = Object.keys(output)
    const firstItem = output[keys[0]]

    if ('messages' in firstItem && Array.isArray(firstItem.messages)) {
        const lastMessage = firstItem.messages[firstItem.messages.length - 1]
        console.dir({
            type: lastMessage._getType(),
            content: lastMessage.content,
            tool_calls: lastMessage.tool_calls,
        })
    }
    if ('sender' in firstItem) {
        console.log({
            sender: firstItem.sender,
        })
    }
}

for await (const output of await streamResults) {
    if (!output?.__end__) {
        prettifyOutput(output)
        console.log('----')
    }
}